In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

df_reviews = pd.read_csv("data/movie_reviews_processed.csv")

training_data, testing_data = train_test_split(
    df_reviews, 
    test_size = 0.25, 
    random_state = 123
)

In [ ]:
#| label: python-regression-model-train
import statsmodels.api as sm
import statsmodels.formula.api as smf

# features = ["review_year_0", "release_year_0",
#   "age_sc", "length_minutes_sc", 
#   "total_reviews_sc", "word_count_sc", 
#   "genre", "gender", 
#   "reviewer_type", "work_status", 
#   "season"]

model_train = smf.ols(
    formula = 'rating ~ review_year_0 \
    + release_year_0 \
    + age_sc \
    + length_minutes_sc \
    + total_reviews_sc \
    + word_count_sc \
    + genre'
    ,
    data = training_data
).fit()